In [3]:
import os
import json
import pandas as pd
import networkx as nx
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import BertTokenizer, BertModel
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GATConv, global_mean_pool
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tqdm import tqdm

In [ ]:
base_dir = r"C:\Users\bruno\rnn_gnn_rumor_detection\rnn_gnn_rumor_detection\PHEME_veracity\all-rnr-annotated-threads"

all_dataframes = []

import pandas as pd
import os
import json

all_dataframes = []

def process_file(file_path, event_folder, rumour_type_folder, thread_id, file_name):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        data['event'] = event_folder
        data['rumour_type'] = rumour_type_folder
        data['thread_id'] = thread_id
        data['file_name'] = file_name

        df_temp = pd.json_normalize(data)
        all_dataframes.append(df_temp)
    except json.JSONDecodeError as e:
        print(f"    Erro de JSON ao ler {file_path}: {e}")
    except Exception as e:
        print(f"    Erro inesperado ao ler {file_path}: {e}")


for event_folder in os.listdir(base_dir):
    event_path = os.path.join(base_dir, event_folder)

    if os.path.isdir(event_path) and not event_folder.startswith('.'):
        print(f"Processando evento: {event_folder}")

        non_rumours_path = os.path.join(event_path, 'non-rumours')
        if os.path.isdir(non_rumours_path):
            print(f"  Entrando em: {non_rumours_path} (lendo TODOS os tweets)")
            for thread_id_folder in os.listdir(non_rumours_path):
                thread_id_path = os.path.join(non_rumours_path, thread_id_folder)

                if os.path.isdir(thread_id_path) and not thread_id_folder.startswith('.'):
                    # Percorre os arquivos JSON dentro de cada ID de thread
                    for file_name in os.listdir(thread_id_path):
                        file_path = os.path.join(thread_id_path, file_name)
                        if os.path.isfile(file_path) and file_name.endswith('.json'):
                            process_file(file_path, event_folder, 'non-rumours', thread_id_folder, file_name)
            

        rumours_path = os.path.join(event_path, 'rumours')
        if os.path.isdir(rumours_path):
            print(f"  Entrando em: {rumours_path} (lendo APENAS 'source_tweet')")
            for thread_id_folder in os.listdir(rumours_path):
                thread_id_path = os.path.join(rumours_path, thread_id_folder)

                if os.path.isdir(thread_id_path) and not thread_id_folder.startswith('.'):
                    source_tweet_path = os.path.join(thread_id_path, 'source_tweet')

                    if os.path.isdir(source_tweet_path):
                        print(f"    Acessando 'source_tweet' em {thread_id_folder}")
                        # Lê os arquivos JSON dentro de 'source_tweet'
                        for file_name in os.listdir(source_tweet_path):
                            file_path = os.path.join(source_tweet_path, file_name)
                            if os.path.isfile(file_path) and file_name.endswith('.json'):
                                process_file(file_path, event_folder, 'rumours', thread_id_folder, file_name)                        
if all_dataframes:
    final_df = pd.concat(all_dataframes, ignore_index=True)
    print("\n---------------------------------------------------")
    print("DataFrame final combinado (primeiras 5 linhas):")
    print(final_df.head())
    print(f"\nTotal de linhas no DataFrame combinado: {len(final_df)}")
    print(f"Colunas do DataFrame: {final_df.columns.tolist()}")
    print("\nEventos únicos no DataFrame:", final_df['event'].unique())
    print("Tipos de rumor únicos no DataFrame:", final_df['rumour_type'].unique())
else:
    print("Nenhum arquivo de dados foi encontrado ou lido na estrutura de pastas com as condições especificadas.")



Processando evento: charliehebdo-all-rnr-threads
  Entrando em: C:\Users\bruno\rnn_gnn_rumor_detection\rnn_gnn_rumor_detection\PHEME_veracity\all-rnr-annotated-threads\charliehebdo-all-rnr-threads\non-rumours (lendo TODOS os tweets)
    Erro inesperado ao ler C:\Users\bruno\rnn_gnn_rumor_detection\rnn_gnn_rumor_detection\PHEME_veracity\all-rnr-annotated-threads\charliehebdo-all-rnr-threads\non-rumours\552784600502915072\._annotation.json: 'utf-8' codec can't decode byte 0xa2 in position 37: invalid start byte
    Erro inesperado ao ler C:\Users\bruno\rnn_gnn_rumor_detection\rnn_gnn_rumor_detection\PHEME_veracity\all-rnr-annotated-threads\charliehebdo-all-rnr-threads\non-rumours\552784600502915072\._structure.json: 'utf-8' codec can't decode byte 0xa2 in position 37: invalid start byte
    Erro inesperado ao ler C:\Users\bruno\rnn_gnn_rumor_detection\rnn_gnn_rumor_detection\PHEME_veracity\all-rnr-annotated-threads\charliehebdo-all-rnr-threads\non-rumours\552784898743099392\._annotation.

KeyboardInterrupt: 

In [13]:
import pandas as pd
import os
import json


base_dir = r"C:\Users\bruno\rnn_gnn_rumor_detection\rnn_gnn_rumor_detection\PHEME_veracity\all-rnr-annotated-threads"

all_dataframes = []

def process_file(file_path, event_folder, rumour_type_folder, thread_id, file_name):
    """Função auxiliar para ler um arquivo JSON e adicionar ao DataFrame."""
    if file_name.startswith('._') or file_name == '.DS_Store':
        return

    encodings_to_try = ['utf-8', 'latin-1', 'cp1252']
    data = None
    for encoding in encodings_to_try:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                data = json.load(f)
            break
        except json.JSONDecodeError as e:
            data = None
        except UnicodeDecodeError as e:
            data = None
        except Exception as e:
            print(f"    Erro inesperado ao ler {file_name} com '{encoding}': {e}")
            data = None
            break

    if data is not None:
        try:
            data['event'] = event_folder
            data['rumour_type'] = rumour_type_folder
            data['thread_id'] = thread_id
            data['file_name'] = file_name 

            df_temp = pd.json_normalize(data)
            all_dataframes.append(df_temp)
        except Exception as e:
            print(f"    Erro ao normalizar JSON de {file_name}: {e}")
    else:
        if not (file_name.startswith('._') or file_name == '.DS_Store'):
            print(f"    Não foi possível ler ou decodificar o arquivo: {file_name} após tentar todas as codificações.")


for event_folder in os.listdir(base_dir):
    event_path = os.path.join(base_dir, event_folder)

    if os.path.isdir(event_path) and not event_folder.startswith('.'):
        print(f"Processando evento: {event_folder}")

        for rumour_type in ['non-rumours', 'rumours']:
            rumour_type_path = os.path.join(event_path, rumour_type)

            if os.path.isdir(rumour_type_path):
                print(f"  Entrando em: {rumour_type_path} (lendo 'source-tweets' JSONs)")

                for thread_id_folder in os.listdir(rumour_type_path):
                    thread_id_path = os.path.join(rumour_type_path, thread_id_folder)

                    if os.path.isdir(thread_id_path) and not thread_id_folder.startswith('.'):
                        source_tweets_folder_path = os.path.join(thread_id_path, 'source-tweets')

                        if os.path.isdir(source_tweets_folder_path):
                            for file_name in os.listdir(source_tweets_folder_path):
                                if os.path.isfile(os.path.join(source_tweets_folder_path, file_name)) and file_name.endswith('.json'):
                                    process_file(os.path.join(source_tweets_folder_path, file_name),
                                                 event_folder, rumour_type, thread_id_folder, file_name)
                                
if all_dataframes:
    final_df = pd.concat(all_dataframes, ignore_index=True)
    print("\n---------------------------------------------------")
    print("DataFrame final combinado (primeiras 5 linhas):")
    print(final_df.head())
    print(f"\nTotal de linhas no DataFrame combinado: {len(final_df)}")
    print(f"Colunas do DataFrame: {final_df.columns.tolist()}")
    print("\nEventos únicos no DataFrame:", final_df['event'].unique())
    print("Tipos de rumor únicos no DataFrame:", final_df['rumour_type'].unique())
else:
    print("Nenhum arquivo de dados foi encontrado ou lido na estrutura de pastas com as condições especificadas.")

Processando evento: charliehebdo-all-rnr-threads
  Entrando em: C:\Users\bruno\rnn_gnn_rumor_detection\rnn_gnn_rumor_detection\PHEME_veracity\all-rnr-annotated-threads\charliehebdo-all-rnr-threads\non-rumours (lendo 'source-tweets' JSONs)
  Entrando em: C:\Users\bruno\rnn_gnn_rumor_detection\rnn_gnn_rumor_detection\PHEME_veracity\all-rnr-annotated-threads\charliehebdo-all-rnr-threads\rumours (lendo 'source-tweets' JSONs)
Processando evento: ebola-essien-all-rnr-threads
  Entrando em: C:\Users\bruno\rnn_gnn_rumor_detection\rnn_gnn_rumor_detection\PHEME_veracity\all-rnr-annotated-threads\ebola-essien-all-rnr-threads\non-rumours (lendo 'source-tweets' JSONs)
  Entrando em: C:\Users\bruno\rnn_gnn_rumor_detection\rnn_gnn_rumor_detection\PHEME_veracity\all-rnr-annotated-threads\ebola-essien-all-rnr-threads\rumours (lendo 'source-tweets' JSONs)
Processando evento: ferguson-all-rnr-threads
  Entrando em: C:\Users\bruno\rnn_gnn_rumor_detection\rnn_gnn_rumor_detection\PHEME_veracity\all-rnr-anno

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)       # Exibir todas as colunas
pd.set_option('display.width', 1000)             # Aumentar largura para tentar manter colunas na mesma linha
pd.set_option('display.max_colwidth', None)      # Não truncar o conteúdo das células
pd.set_option('display.max_rows', 20)            # Exibir mais linhas no head/tail

if 'final_df' in locals() and not final_df.empty:
    print("\n---------------------------------------------------")
    print("DataFrame final combinado (primeiras 10 linhas, formatado):")
    print(final_df.head(10))

    print("\n---------------------------------------------------")
    print("Informações detalhadas do DataFrame:")
    final_df.info() # .info() já imprime e retorna None, não precisa de print() em volta

    print("\n---------------------------------------------------")
    print(f"Total de linhas no DataFrame combinado: {len(final_df)}")
    print("Colunas do DataFrame:", final_df.columns.tolist())
    print("Eventos únicos no DataFrame:", final_df['event'].unique())
    print("Tipos de rumor únicos no DataFrame (diretório):", final_df['rumour_type'].unique())


    output_csv_path = 'tweets_phm_processed_source_tweets.csv'
    try:
        final_df.to_csv(output_csv_path, index=False, encoding='utf-8')
        print(f"\nDataFrame salvo com sucesso em: {output_csv_path}")
    except Exception as e:
        print(f"\nErro ao salvar o DataFrame em CSV: {e}")

else:
    print("\nO DataFrame 'final_df' não existe ou está vazio. Não há nada para visualizar ou exportar.")



---------------------------------------------------
DataFrame final combinado (primeiras 10 linhas, formatado):
  contributors  truncated                                                                                                                                        text in_reply_to_status_id                  id  favorite_count                                                                               source  retweeted coordinates in_reply_to_screen_name              id_str  retweet_count in_reply_to_user_id  favorited   geo in_reply_to_user_id_str lang                      created_at in_reply_to_status_id_str place                         event  rumour_type           thread_id                file_name label entities.symbols                                                                                                     entities.user_mentions                                                                        entities.hashtags  \
0         None      False               

In [20]:
df = pd.read_csv(r"C:\Users\bruno\rnn_gnn_rumor_detection\rnn_gnn_rumor_detection\src\tweets_phm_processed_source_tweets.csv")

import pandas as pd

df = pd.read_csv(r"C:\Users\bruno\rnn_gnn_rumor_detection\rnn_gnn_rumor_detection\src\tweets_phm_processed_source_tweets.csv")

pd.set_option('display.max_columns', None)       # Show all columns
pd.set_option('display.width', 1000)             # Adjust display width
pd.set_option('display.max_colwidth', None)      # Do not truncate column content
pd.set_option('display.max_rows', 10)            # Display up to 10 rows for head/tail

print("--- Displaying 10 examples for each column ---")

for column_name in df.columns:
    print(f"\n--- Examples for column: '{column_name}' ---")
    if len(df) >= 10:
        print(df[column_name].head(10).to_string()) # .to_string() ensures full display of the Series
    else:
        print(df[column_name].to_string()) # If less than 10, display all available rows
    print("-" * (len(column_name) + 25)) # A separator line for clarity

--- Displaying 10 examples for each column ---

--- Examples for column: 'contributors' ---
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
6   NaN
7   NaN
8   NaN
9   NaN
-------------------------------------

--- Examples for column: 'truncated' ---
0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
----------------------------------

--- Examples for column: 'text' ---
0                                                            Charlie Hebdo became well known for publishing the Muhammed cartoons two years ago
1                                                     Charlie Hebdo’s Last Tweet Before Shootings http://t.co/9Oa2xAqOcM http://t.co/skJHNEQcn0
2                                               Prediction: the #CharlieHebdo massacre will not dent the political class's complacency one iota
3      10:28am Charlie Hebdo account mocks ISIS leader, wishing him "good health and best wishes" http://t.co/9Oa2xAqOcM http://t.co/u

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

if 'df' not in locals() or df.empty:
    print("Erro: O DataFrame 'df' não foi carregado ou está vazio. Execute a Célula 1 primeiro.")
else:
    X = df['text'].astype(str)
    y = df['label']

    print(f"\nNúmero total de amostras: {len(X)}")
    print(f"Primeiras 5 amostras de X (text):\n{X.head()}")
    print(f"\nPrimeiras 5 amostras de y (label):\n{y.head()}")
    print(f"\nDistribuição dos rótulos:\n{y.value_counts()}")

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    print(f"\nRótulos codificados (primeiras 5):\n{y_encoded[:5]}")
    print(f"Mapeamento de rótulos: {list(label_encoder.classes_)} -> {label_encoder.transform(label_encoder.classes_)}")

    max_words = 10000
    maxlen = 100

    tokenizer = Tokenizer(num_words=max_words, oov_token="<unk>")
    tokenizer.fit_on_texts(X)

    X_sequences = tokenizer.texts_to_sequences(X)
    X_padded = pad_sequences(X_sequences, maxlen=maxlen, padding='post', truncating='post')

    print(f"\nComprimento médio da sequência original: {sum(len(s) for s in X_sequences) / len(X_sequences):.2f}")
    print(f"Exemplo da primeira sequência (antes do padding):\n{X_sequences[0]}")
    print(f"Exemplo da primeira sequência (após padding/truncamento):\n{X_padded[0]}")
    print(f"Formato dos dados de entrada para o modelo: {X_padded.shape}")

    X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

    print(f"\nFormato do conjunto de treino (X_train): {X_train.shape}")
    print(f"Formato do conjunto de teste (X_test): {X_test.shape}")
    print(f"Formato dos rótulos de treino (y_train): {y_train.shape}")
    print(f"Formato dos rótulos de teste (y_test): {y_test.shape}")

    train_labels_counts = pd.Series(y_train).value_counts().sort_index()
    test_labels_counts = pd.Series(y_test).value_counts().sort_index()
    print("\nDistribuição de rótulos no treino:")
    for label_val, count in train_labels_counts.items():
        print(f"  {label_encoder.inverse_transform([label_val])[0]} ({label_val}): {count}")
    print("Distribuição de rótulos no teste:")
    for label_val, count in test_labels_counts.items():
        print(f"  {label_encoder.inverse_transform([label_val])[0]} ({label_val}): {count}")

    X_train_tensor = torch.LongTensor(X_train)
    y_train_tensor = torch.LongTensor(y_train)
    X_test_tensor = torch.LongTensor(X_test)
    y_test_tensor = torch.LongTensor(y_test)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    batch_size = 64
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    class RumourRNN(nn.Module):
        def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, dropout_rate):
            super().__init__()
            
            self.embedding = nn.Embedding(vocab_size, embedding_dim)
            self.lstm = nn.LSTM(embedding_dim,
                                hidden_dim,
                                num_layers=num_layers,
                                bidirectional=False,
                                dropout=dropout_rate if num_layers > 1 else 0,
                                batch_first=True)
            
            self.dropout = nn.Dropout(dropout_rate)
            self.fc = nn.Linear(hidden_dim, output_dim)
            
        def forward(self, text):
            
            embedded = self.dropout(self.embedding(text))
            
            output, (hidden, cell) = self.lstm(embedded)
            
            final_hidden_state = hidden[-1,:,:]
            
            output = self.fc(self.dropout(final_hidden_state))
            return output

    vocab_size = max_words
    embedding_dim = 128
    hidden_dim = 128
    output_dim = num_classes
    num_layers = 2
    dropout_rate = 0.5

    model = RumourRNN(vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, dropout_rate)

    print("\nEstrutura do modelo RNN definida (PyTorch):")
    print(model)

    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    criterion = criterion.to(device)

    def binary_accuracy(preds, y):
        if preds.shape[1] > 1:
            _, predicted_labels = torch.max(preds, 1)
        else:
            predicted_labels = (preds > 0.5).long()
        correct = (predicted_labels == y).float()
        acc = correct.sum() / len(correct)
        return acc

    epochs = 30
    print(f"\nIniciando o treinamento do modelo PyTorch em {device}...")
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        epoch_acc = 0

        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            optimizer.zero_grad()
            
            predictions = model(batch_X)
            loss = criterion(predictions, batch_y)
            acc = binary_accuracy(predictions, batch_y)
            
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += acc.item()

        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {epoch_loss/len(train_loader):.4f} - Train Acc: {epoch_acc/len(train_loader):.4f}")

    model.eval()
    test_loss = 0
    test_acc = 0

    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            predictions = model(batch_X)
            loss = criterion(predictions, batch_y)
            acc = binary_accuracy(predictions, batch_y)

            test_loss += loss.item()
            test_acc += acc.item()

    print("\nTreinamento concluído. Avaliando o modelo no conjunto de teste (PyTorch)...")
    print(f"Acurácia no conjunto de teste: {(test_acc/len(test_loader))*100:.2f}%")
    print(f"Loss no conjunto de teste: {test_loss/len(test_loader):.4f}")
    


Número total de amostras: 6425
Primeiras 5 amostras de X (text):
0                                                           Charlie Hebdo became well known for publishing the Muhammed cartoons two years ago
1                                                    Charlie Hebdo’s Last Tweet Before Shootings http://t.co/9Oa2xAqOcM http://t.co/skJHNEQcn0
2                                              Prediction: the #CharlieHebdo massacre will not dent the political class's complacency one iota
3     10:28am Charlie Hebdo account mocks ISIS leader, wishing him "good health and best wishes" http://t.co/9Oa2xAqOcM http://t.co/uYXayKLA7q
4    If your faith isn't strong enough to cope with satirical poke, it oughtn't be strong enough to induce you to kill. Barbaric #CharlieHebdo
Name: text, dtype: object

Primeiras 5 amostras de y (label):
0    True
1    True
2    True
3    True
4    True
Name: label, dtype: bool

Distribuição dos rótulos:
label
True     4023
False    2402
Name: count, dtype: i